# CS 634 Data Mining Midterm Project


In [1]:
import numpy as np
import pandas as pd

In [2]:
def distinct(transaction):  ## input a list of set
    lis = []
    
    for transac in transaction:
        for item in transac:
            if not item in lis:
                lis.append(item)

    return set(lis)  ## return a set of distinct items

In [3]:
from itertools import combinations
def generate(lis, n):  ## input a list of itemset and number of items in new sets
    candidate = []
    
    for i in range(len(lis)):
        for j in range(i+1, len(lis)):
            union = lis[i].union(lis[j])
            if len(union) == n and (not union in candidate):
                candidate.append(union)
                
    return candidate  ## return a candidate list with candidate size n from prior list

In [4]:
def support(transaction, itemset, minisupp):  ## input a list of set, list of set, num
    freq = [0]*len(itemset)
    for i in range(len(freq)):
        for j in range(len(transaction)):
            if itemset[i].issubset(transaction[j]):
                freq[i] += 1
                
        freq[i] = freq[i]/len(transaction)
        
    df = {'itemset': itemset, 'freq': freq}
    res = pd.DataFrame(df)
    
    return res[res.freq >= minisupp]  ## return a df of itemsets with larger than minisupp

In [5]:
def apriori(transaction, minisupp):  ## input a list of set and a value
    lis = list(distinct(transaction))
    lis = list(set([i]) for i in lis)
    candidate = support(list(transaction), lis, minisupp)
    result = pd.DataFrame()

    k = 2
    while (candidate.empty == False):
        lis = generate(list(candidate.itemset), k)
        candidate = support(list(transaction), list(lis), minisupp)
        result = result.append(candidate)
        k += 1

    return result  ## return a df of frequent set with support


In [6]:
import itertools
def association(freqset):  ## input the requent set df
    d = {'known':[], 'inference':[]}
    rule = pd.DataFrame(d)
    freq = list(freqset.itemset)
    
    for i in range(len(freq)):
        if len(freq[i])>1:
            templis = freq[i]
        for m in range(1, len(templis)):
            known = map(set, itertools.combinations(templis, m))
            known = list(known)
            for j in range(len(known)):
                inference = templis - known[j]
                d = [inference, known[j]]
            
                rule.loc[len(rule)]=d
            
    return rule  ## return the association rules
            

In [7]:
def frequency(itemset, transaction):  ## input itemset or item and list of set for transaction
    count = 0
    for i in range(len(transaction)):
        if itemset.issubset(transaction[i]):
            count += 1
       
    return count  ## return the frequency for a single itemset or item

In [8]:
def confidence(rule, transaction, miniconf):  ## input df association rule, list of set for transaction and a val
    conf = [0]*len(rule)
    inference = list(rule.inference)
    known = list(rule.known)
    
    for i in range(len(rule)):
        tempset = inference[i].union(known[i])
        setfreq = frequency(tempset, transaction)
        knownfreq = frequency(known[i], transaction)
        
        conf[i] = setfreq/knownfreq
        
    rule['confidence'] = conf
    
    return rule[rule.confidence>=miniconf]  ## return association rules with confidence

In [14]:
data = pd.read_csv('record1.txt', sep=' ', header=None, 
                   names = ('transaction', 'itemset'))

for i in range(len(data.itemset)):  ## data preprocessing 
    data.itemset[i] = data.itemset[i].split(',')  ## split items by commas
    data.itemset[i] = set(data.itemset[i])  ## convert list to list of set

data

/home/shiboyao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/shiboyao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,transaction,itemset
0,101,"{keyboard, folder, pad, mouse, laptop, wire}"
1,102,"{mouse, pad, keyboard, monitor}"
2,103,"{pad, laptop, folder, case}"
3,104,"{monitor, keyboard, usb, book, mouse, wire}"
4,105,"{wire, monitor, usb, keyboard}"
5,106,"{folder, monitor, keyboard, pad, usb, book, ca..."
6,107,"{wire, laptop, monitor}"
7,108,"{laptop, case}"
8,109,"{folder, keyboard, book, case, mouse, laptop}"
9,110,"{mouse, pad, wire}"


In [15]:
minisupp = float(input('Enter minimum support value:\n '))
miniconf = float(input('Enter minimum confidence value:\n'))


Enter minimum support value:
 0.5
Enter minimum confidence value:
0.5


In [16]:
freqset = apriori(data.itemset, minisupp)
freqset

,itemset,freq
0,"{mouse, keyboard}",0.5


In [17]:
rule = association(freqset)
print(rule)

        known   inference
0  {keyboard}     {mouse}
1     {mouse}  {keyboard}


In [18]:
confidence(rule, data.itemset, miniconf)

,known,inference,confidence
0,{keyboard},{mouse},0.833333
1,{mouse},{keyboard},0.833333
